# IEEE 802.16 Completed Projects

In [161]:
import mechanicalsoup
import bs4
from bs4 import BeautifulSoup
import re
import string
import xlwt

In [18]:
# Connect to IEEE 802.16 Completed Projects
browser = mechanicalsoup.StatefulBrowser()
root1 = "http://www.ieee802.org/16/"
url = "http://www.ieee802.org/16/tgs.html"
browser.open(url)

<Response [200]>

### Crawler web link down

In [4]:
# Get html content
html = browser.get_current_page()

In [42]:
blocks = html.find_all('ul')[1:5]
## feature1 && feature2
key = ['Air Interface', 'Bridging', 'Conformance', 'Coexistence']
fea12Dic = {'Air Interface': 
                {
                    'link': [], 'feature2': []
                }, 
            'Bridging': 
                {
                    'link': [], 'feature2': []
                }, 
            'Conformance': 
                {
                    'link': [], 'feature2': []
                }, 
            'Coexistence': 
                {
                    'link': [], 'feature2': []
                }
           }
for idx1, block in zip(range(len(blocks)), blocks):
    lis = block.find_all('li')
    for idx2, li in zip(range(len(lis)), lis):
        a = li.a
        link = root1 + a['href']
        fea12Dic[key[idx1]]['link'].append(link)
        if idx1 == len(blocks)-1 and idx2 == 0:
            feature2 = li.contents[-3].strip().strip(string.punctuation)
        else:
            feature2 = li.contents[-1].strip().strip(string.punctuation)
        fea12Dic[key[idx1]]['feature2'].append(feature2)

In [333]:
fea12Dic

{'Air Interface': {'link': ['http://www.ieee802.org/16/tg1/index.html',
   'http://www.ieee802.org/16/tga/index.html',
   'http://www.ieee802.org/16/tgc/index.html',
   'http://www.ieee802.org/16/tgd/index.html',
   'http://www.ieee802.org/16/netman/16f.html',
   'http://www.ieee802.org/16/maint/index.html',
   'http://www.ieee802.org/16/tge/index.html',
   'http://www.ieee802.org/16/netman/16g.html',
   'http://www.ieee802.org/16/maint/index.html',
   'http://www.ieee802.org/16/relay/index.html',
   'http://www.ieee802.org/16/le/index.html',
   'http://www.ieee802.org/16/tgm/index.html',
   'http://www.ieee802.org/16/maint/index.html',
   'http://www.ieee802.org/16/maint/index.html',
   'http://www.ieee802.org/16/m2m/index.html',
   'http://www.ieee802.org/16/m2m/index.html',
   'http://www.ieee802.org/16/gridman/index.html',
   'http://www.ieee802.org/16/gridman/index.html',
   'http://www.ieee802.org/16/16q/index.html',
   'http://www.ieee802.org/16/gridman/index.html',
   'http://w

### Crawler each link

In [361]:
res = []
total = len(fea12Dic)
for idx, feature1 in zip(range(total), fea12Dic.keys()):
    print("%d/%d"%(idx+1, total), end="\r")
    for url, feature2 in zip(fea12Dic[feature1]['link'], fea12Dic[feature1]['feature2']):
        browser.open(url)
        html = browser.get_current_page()
        prefix = url[0:url.rfind('/')+1]
        # each page
        for tag in html.find_all('b', href=False):    
            tagtext = tag.text
            if "Contributed" in tagtext or "Contributions" in tagtext:
                for ul in tag.find_next_siblings('ul', class_=False):
                    # Session Info
                    if ul.b is not None:
                        sessionInfo = ul.b.text
                    elif ul.strong is not None:
                        sessionInfo = ul.strong.text
                    else:
                        sessionInfo = None
                    for li in ul.find_all('li', class_=False):
                        if li.a is not None:
                            [time, title, author, fileLink, addiInfo] = parse(li, prefix)
                            res.append([feature1, feature2, tagtext, sessionInfo, title, author, time, fileLink, addiInfo])
            elif "Official" in tagtext or tagtext == "Documents" or "Task" in tagtext:
                # Session Info
                sessionInfo = None
                if type(tag.parent.find_next_sibling('ul')) is not bs4.element.NavigableString and tag.parent.find_next_sibling('ul') is not None:
                    tmp = tag.parent
                else:
                    tmp = tag
                for li in tmp.find_all('li'):
                    if li.a is not None:
                        [time, title, author, fileLink, addiInfo] = parse(li, prefix)
                        res.append([feature1, feature2, tagtext, sessionInfo, title, author, time, fileLink, addiInfo])
        for tag in html.find_all('strong', href=False):
            tagtext = tag.text
            if "Contributed" in tagtext or "Contributions" in tagtext:
                for ul in tag.parent.find_next_siblings('ul', class_=False):
                    # Session Info
                    if ul.b is not None:
                        sessionInfo = ul.b.text
                    elif ul.strong is not None:
                        sessionInfo = ul.strong.text
                    else:
                        sessionInfo = None
                    for li in ul.find_all('li', class_=False):
                        if li.a is not None:
                            [time, title, author, fileLink, addiInfo] = parse(li, prefix)
                            res.append([feature1, feature2, tagtext, sessionInfo, title, author, time, fileLink, addiInfo])
            elif "Official" in tagtext or tagtext == "Documents" or "Task" in tagtext:
                # Session Info
                sessionInfo = None
                if type(tag.parent.find_next_sibling('ul')) is not bs4.element.NavigableString and tag.parent.find_next_sibling('ul') is not None:
                    tmp = tag.parent
                else:
                    tmp = tag
                for li in tmp.find_all('li'):
                    if li.a is not None:
                        [time, title, author, fileLink, addiInfo] = parse(li, prefix)
                        res.append([feature1, feature2, tagtext, sessionInfo, title, author, time, fileLink, addiInfo])

In [365]:
len(res)

17282

### Util Functions

In [360]:
def parse(li, prefix):
    conts = li.contents
    tmpstr = conts[-1]
    if tmpstr.find('(') == None or tmpstr.find(')') == None:
        authAndTimeCtx = ""
    else:
        authAndTimeCtx = tmpstr[tmpstr.find('(')+1:tmpstr.find(')')]
    # File link
    links = li.find_all('a', href=True)
    if links == []:
        li.find_all('a', nhref=True)
    if links == []:
        fileLink = None
    else:
        fileLink = prefix + links[0]['href']
    # title
    title = ""
    flag = False
    for cont in conts:
        if flag and cont.string is not None:
            title += cont.string
        elif cont.string is None:
            flag = True
    if title == "":
        title = None
    else:
        title = title.strip()
    # time
    time = None
    target = -1
    for idx in range(len(conts)-1, -1, -1):
        mat1 = re.search(r"(\d{4}-\d{2}-\d{2})", str(conts[idx]))
        if mat1 is not None:
            time = mat1.group(0)
            target = idx
            break
        else:
            mat2 = re.search(r"(\d{2}/\d{2}/\d{2})", str(li.text))
            if mat2 is not None:
                time = mat2.group(0)
                time = time.replace("/", "-");
                year = int(time[0:2])
                if year >= 80:
                    time = "19" + time
                else:
                    time = "20" + time
                target = idx
                break
    # author
    if target == -1:
        author = None
    else:
        tmpsplit = re.split('[;,]', str(conts[target]))
        author = ""
        for ele in tmpsplit:
            mat1 = re.search(r"(\d{4}-\d{2}-\d{2})", ele)
            mat2 = re.search(r"(\d{2}/\d{2}/\d{2})",ele)
            if mat1 is None and mat2 is None:
                author += ele + ";"
        if author is "":
            author = None
    # addiInfo
    addiInfo = ""
    for idx in range(1, target):
        tmp = str(conts[idx])
        if tmp != '<br/>':
            addiInfo += tmp
    if addiInfo is "":
        addiInfo = None
    return [time, title, author, fileLink, addiInfo]

### Save list into excel file

In [367]:
filename = ""
book = xlwt.Workbook(encoding="utf-8")
sheet1 = book.add_sheet("Sheet 1")
sheet1.write(0, 0, "Feature1")
sheet1.write(0, 1, "Feature2")
sheet1.write(0, 2, "Tag Text")
sheet1.write(0, 3, "Session Info")
sheet1.write(0, 4, "Title")
sheet1.write(0, 5, "Author")
sheet1.write(0, 6, "Time")
sheet1.write(0, 7, "File Link")
sheet1.write(0, 8, "Additional Infoamtion")

In [368]:
# [feature1, feature2, tagtext, sessionInfo, title, author, time, fileLink, addiInfo]
for idx1, line in zip(range(1, len(res)+1), res):
    for idx2, val in zip(range(0, len(line)), line):
        sheet1.write(idx1, idx2, val)

In [369]:
book.save("802_16_Completed_Projects.xls")

In [ ]:
str1 = "<li><a href='contrib/802161c-01_39.pdf'>802.16.1c-01/39</a> (2001-09-13)<br/>Comment Resolution Details (<a href='mailto:carl.eklund@nokia.com.com'>Carl Eklund</a>)</li>"
str2 = "<li><a href='11/80216-11_0041r3.zip' name='11_0041'>IEEE 802.16-11/0041r3</a> (<a href='11/80216-11_0041r2.zip'>IEEE 802.16-11/0041r2</a>: 2011-11-10; <a href='11/80216-11_0041r1.zip'>IEEE 802.16-11/0041r1</a>: 2011-11-08; <a href='11/80216-11_0041.zip'>IEEE 802.16-11/0041</a>: 2011-11-07)<br/> IEEE 802.16 WG Letter Ballot #34 Comments (99/11/16)</li>"
soup = BeautifulSoup(str1)
parse(soup.li,"")

In [ ]:
http://www.ieee802.org/16/docs/06/C80216-06_001.pdf

In [ ]:
http://www.ieee802.org/16/tgs.html
http://www.ieee802.org/16/tg1/index.html
http://www.ieee802.org/16/tg1/docs/802161-01_08.pdf

In [282]:
url = "http://www.ieee802.org/16/tgc/C1/index.html"
browser.open(url)
html = browser.get_current_page()
root = url[0:url.rfind('/')+1]
# each page
test = []
prefix = ""
for tag in html.find_all('b', href=False):    
    tagtext = tag.text
    if "Contributed" in tagtext or "Contributions" in tagtext:
        for ul in tag.find_next_siblings('ul', class_=False):
            # Session Info
            if ul.b is not None:
                sessionInfo = ul.b.text
            elif ul.strong is not None:
                sessionInfo = ul.strong.text
            else:
                sessionInfo = None
            for li in ul.find_all('li', class_=False):
                if li.a is not None:
                    [time, title, author, fileLink, addiInfo] = parse(li, prefix)
                    res.append([feature1, feature2, tagtext, sessionInfo, title, author, time, fileLink, addiInfo])
    elif "Official" in tagtext or tagtext == "Documents" or "Task" in tagtext:
        # Session Info
        sessionInfo = None
        if type(tag.parent.find_next_sibling('ul')) is not bs4.element.NavigableString and tag.parent.find_next_sibling('ul') is not None:
            tmp = tag.parent
            print("1")
        else:
            tmp = tag
            print("2")
        for li in tmp.find_all('li'):
            if li.a is not None:
                [time, title, author, fileLink, addiInfo] = parse(li, prefix)
                res.append([feature1, feature2, tagtext, sessionInfo, title, author, time, fileLink, addiInfo])
for tag in html.find_all('strong', href=False):
    tagtext = tag.text
    if "Contributed" in tagtext or "Contributions" in tagtext:
        for ul in tag.parent.find_next_siblings('ul', class_=False):
            # Session Info
            if ul.b is not None:
                sessionInfo = ul.b.text
            elif ul.strong is not None:
                sessionInfo = ul.strong.text
            else:
                sessionInfo = None
            for li in ul.find_all('li', class_=False):
                if li.a is not None:
                    [time, title, author, fileLink, addiInfo] = parse(li, prefix)
                    res.append([feature1, feature2, tagtext, sessionInfo, title, author, time, fileLink, addiInfo])
    elif "Official" in tagtext or tagtext == "Documents" or "Task" in tagtext:
        # Session Info
        sessionInfo = None
        if type(tag.parent.find_next_sibling('ul')) is not bs4.element.NavigableString and tag.parent.find_next_sibling('ul') is not None:
            tmp = tag.parent
        else:
            tmp = tag
        for li in tmp.find_all('li'):
            if li.a is not None:
                [time, title, author, fileLink, addiInfo] = parse(li, prefix)
                res.append([feature1, feature2, tagtext, sessionInfo, title, author, time, fileLink, addiInfo])

2
<li><a href="contrib/C1802161-02_01.pdf">C1802.16.1-02/01</a><br/>Protocol Implementation Conformance Statement for IEEE Standard 802.16-2001 (<a href="mailto:ken@ensemble.com">Ken Stanwood</a>, 02/09/17)<br/></li>
[<a href="contrib/C1802161-02_01.pdf">C1802.16.1-02/01</a>, <br/>, 'Protocol Implementation Conformance Statement for IEEE Standard 802.16-2001 (', <a href="mailto:ken@ensemble.com">Ken Stanwood</a>, ', 02/09/17)', <br/>]
<br/>


AttributeError: 'NoneType' object has no attribute 'strip'

In [209]:
len(res)

2443

In [ ]:
==Documents or Contains Contributions or Contains Official or Contains Task

In [207]:
a = None
a == None

True

In [164]:
type(tag.parent.next_sibling) is bs4.element.NavigableString

True

In [167]:
tag.find_next_sibling('ul').find_all('li')

[<li><a href="docs/802161-01_08.pdf">802.16.1-01/08</a><br/>
     IEEE 802.16 TG1 MAC Meeting Minutes for Session #12 (01/03/21)
 	</li>, <li><a href="docs/802161-01_07.pdf">802.16.1-01/07</a><br/>
 	IEEE 802.16.1 Action Report on Comment Resolution at Session #11 (01/02/18)
 	</li>, <li><a href="docs/802161-01_06.pdf">802.16.1-01/06</a><br/>
     IEEE 802.16 MAC &amp; Task Group 1 Meeting Minutes for Session #11 (01/01/22)
 	</li>, <li><a href="docs/802161-01_05.pdf">802.16.1-01/05</a><br/>
     Session #11 TG1 PHY minutes (01/01/26)
 	</li>, <li><a href="docs/802161-01_04.pdf">802.16.1-01/04</a><br/>
     IEEE 802.16.1 Comment Resolutions at Session #11 (01/01/26)
 	</li>, <li><a href="docs/802161-01_03.pdf">802.16.1-01/03</a><br/>
     IEEE 802.16.1 Comments for Session #11 (<a href="mailto:r.b.marks@ieee.org">Roger Marks</a>, 01/01/17)
 	</li>, <li><a href="docs/802161-01_02.pdf">802.16.1-01/02</a><br/>
     Call for Comments on IEEE 802.16.1/D1-2000 ("Draft Standard for Air Interf

In [ ]:
test =  'http://www.ieee802.org/16/tgc/C2/index.html'
test[0:test.rfind('/')+1]